# Hi :)

----

# Import libraries

In [9]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

# ITG

# Loading the dataset

In [10]:
ITG_features_df= pd.read_csv('Dataset/ITG_features_df.csv')

# Bagging

In [11]:
# Step 1: Extract features and labels
X= ITG_features_df[['F1_sum_vector_magnitude', 'F2_orientation_angle', 'F3_std_magnitude']].values
y= ITG_features_df['Situation'].values

# Step 2: Encode labels ('Fall' → 1, 'Not Fall' → 0)
label_encoder= LabelEncoder()
y_encoded= label_encoder.fit_transform(y)

# Step 3: Split dataset
X_train, X_test, y_train, y_test= train_test_split(X, y_encoded, test_size= 0.2, random_state= 48, stratify= y_encoded)

In [ ]:
# Step 4: Train Bagging
# Define base classifiers with chosen hyperparameters
svm_clf= SVC(kernel= 'rbf', C= 100, gamma= 1, random_state= 48)
rf_clf= RandomForestClassifier(n_estimators= 100, max_depth= 10, min_samples_split= 5, min_samples_leaf= 1, random_state= 48)
xgb_clf= XGBClassifier(n_estimators= 200, max_depth= 5, learning_rate= 0.1, subsample= 1.0, objective= 'binary:logistic', use_label_encoder= False, eval_metric= 'logloss', random_state= 48)

# Function to create a bootstrap sample from training data
def bootstrap_sample(X, y):
    return resample(X, y, replace= True, n_samples= len(y), random_state= None)

# Bootstrap sampling and training for each classifier
X_svm, y_svm= bootstrap_sample(X_train, y_train)
svm_clf.fit(X_svm, y_svm)

X_rf, y_rf = bootstrap_sample(X_train, y_train)
rf_clf.fit(X_rf, y_rf)

X_xgb, y_xgb = bootstrap_sample(X_train, y_train)
xgb_clf.fit(X_xgb, y_xgb)

# Predict test set for each classifier
y_pred_svm = svm_clf.predict(X_test)
y_pred_rf = rf_clf.predict(X_test)
y_pred_xgb = xgb_clf.predict(X_test)

# Majority voting function
def majority_vote(predictions):
    """
    predictions: list of np.array, each with predicted labels
    returns: np.array of majority voted predictions
    """
    preds_stack = np.vstack(predictions)
    majority_pred, _ = mode(preds_stack, axis=0)
    return majority_pred.ravel()

# Combine predictions by majority vote
y_pred_ensemble = majority_vote([y_pred_svm, y_pred_rf, y_pred_xgb])

# Evaluate ensemble accuracy
accuracy = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble accuracy with bootstrap + majority voting: {accuracy:.4f}")
